
- food-order-understanding-small-3200.json (학습)
- food-order-understanding-small-800.json (검증)
- 로깅을 위한 wandb

mistral model
https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

# 1. 세팅

In [ ]:
pip install transformers peft accelerate optimum bitsandbytes trl wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional
import re

import torch
import tyro
from accelerate import Accelerator
from datasets import load_dataset, Dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)

from trl import SFTTrainer

from trl.trainer import ConstantLengthDataset

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

드라이브 마운트 후 파일 업로드
- food-order-understanding-small-3200.json
- food-order-understanding-small-800.json

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# 2. 매개 변수 설정
- 어텐션 메커니즘 참고: https://github.com/mistralai/mistral-src/blob/main/mistral/model.py

In [ ]:
@dataclass
class ScriptArguments:
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "the cache dir"}
    )
    model_name: Optional[str] = field(
        default="mistralai/Mistral-7B-Instruct-v0.2", metadata={"help": "the model name"}
    )

    dataset_name: Optional[str] = field(
        default=None,
        metadata={"help": "the dataset name"},
    )
    seq_length: Optional[int] = field(
        default=1024, metadata={"help": "the sequence length"}
    )
    num_workers: Optional[int] = field(
        default=8, metadata={"help": "the number of workers"}
    )
    training_args: TrainingArguments = field(
        default_factory=lambda: TrainingArguments(
            output_dir="./results",
            # max_steps=500,
            logging_steps=20,
            # save_steps=10,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=2,
            gradient_checkpointing=False,
            group_by_length=False,
            learning_rate=1e-4,
            lr_scheduler_type="cosine",
            # warmup_steps=100,
            warmup_ratio=0.03,
            max_grad_norm=0.3,
            weight_decay=0.05,
            save_total_limit=20,
            save_strategy="epoch",
            num_train_epochs=1,
            optim="paged_adamw_32bit",
            fp16=True,
            remove_unused_columns=False,
            report_to="wandb",
        )
    )

    packing: Optional[bool] = field(
        default=True, metadata={"help": "whether to use packing for SFTTrainer"}
    )

    peft_config: LoraConfig = field(
        default_factory=lambda: LoraConfig(
            r=8,
            lora_alpha=16,
            lora_dropout=0.05,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "down_proj", "up_proj", "gate_proj"],
            bias="none",
            task_type="CAUSAL_LM",
        )
    )

    merge_with_final_checkpoint: Optional[bool] = field(
        default=False, metadata={"help": "Do only merge with final checkpoint"}
    )

# 3. 유틸리티

In [ ]:
def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# 4. 데이터 로딩

수정

---


- Hugging Face의 mistral instruct format 예시에서는 사용자와 어시스턴트 간의 상호작용을 나타내는 채팅형 포맷을 보여주고 있음. 이는 다양한 대화형 태스크에서 매우 유용할 수 있으나, nsmc 데이터셋의 경우는 주어진 리뷰 텍스트에 대한 긍정/부정 분류가 주된 작업이므로, 어시스턴트의 대답을 생성하는 대신 리뷰 텍스트를 분석하여 긍정 또는 부정을 판단하는 것이 목표임.

- `<s>` 스페셜 토큰 관련
  - mistral 모델의 경우, 입력 포맷을 특정 지시문 형태로 구성할 때 명시적으로 `<s>` 토큰을 사용하지 않고, 대신 [INST]와 [/INST] 토큰으로 지시문을 구분하는 방식을 채택 가능.
  - 모델이 내부적으로 텍스트의 시작과 끝을 인식하는 방식에 따라 다르므로, 명시적인 `<s>` 사용은 모델의 입력 처리 방식과 지시문 형식에 따라 결정

- 긍정/부정 분류 명령어 관련
  - mistral 모델을 사용할 때, 특히 instruction based 모델을 사용하는 경우, 모델에 직접적으로 "이 리뷰가 긍정인지 부정인지 분류하라"는 명령어를 입력 텍스트에 포함시키지 않고, 대신 리뷰 텍스트를 모델에 제공하며 모델의 학습된 패턴을 이용해 긍정 또는 부정을 판단하도록 할 수 있음
  -  mistral 모델을 사용하여 nsmc 데이터셋의 리뷰를 긍정 또는 부정으로 분류할 때, 모델에게 직접적인 지시문을 제공하기보다는 리뷰 텍스트를 입력으로 제공하고 모델이 해당 텍스트를 분석하여 긍정 또는 부정으로 분류하도록 함. 이 접근 방식은 모델이 지시문을 해석하고 적절한 출력을 생성할 수 있도록 하는 Hugging Face의 instruction fine-tuning 기능을 활용

- System 명령어 제거 이유
  - System 부분은 사용자와 어시스턴트 간의 상호작용을 나타내는 채팅형 포맷에서 주로 사용되며, 사용자에게 정보를 제공하거나 특정 작업을 지시할 때 유용
  -  NSMC 데이터셋을 사용하여 리뷰의 긍정 또는 부정을 판단하는 작업에 있어서, 리뷰 텍스트 자체와 그에 대한 모델의 분석이 주된 초점
  - System 부분을 통해 추가적인 지시를 제공하기보다는, 리뷰 텍스트(User)만을 사용하여 모델에게 분석 작업을 명확하게 지시하는 것이 목적에 더 부합


In [ ]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset using mistral instruct format."""

    # Mistral Instruct 포맷에 맞게 수정된 프롬프트 템플릿
    # User의 리뷰를 기반으로 긍정 또는 부정을 판단하는 지시문을 생성
    prompt_template = """[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "{User}" [/INST]\n반응: "{Assistant}" """ # 학습
    # prompt_template = """[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "{User}" [/INST]\n반응: """ # 테스트


    text = prompt_template.format(User=example["document"],
                                  Assistant="긍정" if example["label"]==1 else "부정")

    return text

### nsmc 데이터셋을 로드하는 부분
- train_data는 2000개 로드
- test_dataA는 1000개 로드
- 각각 for문 설정

In [ ]:
from datasets import load_dataset

def create_datasets(tokenizer, args):
    # 'nsmc' 데이터셋 로드
    nsmc_dataset = load_dataset('nsmc')

    # 학습 데이터셋 선택
    if 'train' in nsmc_dataset:
        train_data = nsmc_dataset['train'].shuffle(seed=42).select([i for i in range(2000)])
    else:
        train_data = nsmc_dataset.shuffle(seed=42).select([i for i in range(2000)])

    # 문자-토큰 비율 계산
    chars_per_token = chars_token_ratio(train_data, tokenizer)
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

    # 학습 데이터셋 생성
    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=args.seq_length,
        chars_per_token=chars_per_token,
    )

    return train_dataset

# 5. 미세 튜닝용 모델 로딩

In [ ]:
script_args = ScriptArguments(
    num_workers=2,
    seq_length=256,
    dataset_name='/gdrive/MyDrive/nlp/food-order-understanding-small-3200.json',
    model_name='mistralai/Mistral-7B-instruct-v0.2',
    )

세부적인 과정을 보고자 step을 50으로 세분화

In [ ]:
script_args.training_args.logging_steps = 50
script_args.training_args.max_steps = 2000
script_args.training_args.output_dir = '/gdrive/MyDrive/undergraduateReasearcher/mistral-instruct-v0.2-results-nsmc-consecutive'
script_args.training_args.run_name = 'mistral-instruct-v0.2-nsmc-2k-steps'

In [ ]:
print(script_args)

ScriptArguments(cache_dir=None, model_name='mistralai/Mistral-7B-instruct-v0.2', dataset_name='/gdrive/MyDrive/nlp/food-order-understanding-small-3200.json', seq_length=256, num_workers=2, training_args=TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_back

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    script_args.training_args.output_dir, #script_args.model_name,
    quantization_config=bnb_config,
    device_map="auto",  # {"": Accelerator().local_process_index},
    trust_remote_code=True,
    use_auth_token=True,
    cache_dir=script_args.cache_dir,
    torch_dtype=torch.float16
)
base_model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
base_model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=Fa

In [ ]:
peft_config = script_args.peft_config

In [ ]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=8, target_modules={'gate_proj', 'up_proj', 'v_proj', 'down_proj', 'q_proj', 'k_proj', 'o_proj'}, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    script_args.model_name,
    trust_remote_code=True,
    cache_dir=script_args.cache_dir,
)

if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

base_model.config.pad_token_id = tokenizer.pad_token_id

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

`<s>`, `</s>` 스페셜 토큰 첨부 여부 확인
- tokenizer를 통해 예시 리뷰의 정보를 출력하여 모델이 기본적으로 bos_token과 eos_token 정보를 생성하는지 확인 필요
- 이에 따른 프롬프트 명령어 토큰 조정 필요(mistral을 비롯한 모든 LLM Model에 적용되는 사항)

In [ ]:
tokenizer.bos_token_id, tokenizer.eos_token_id

(1, 2)

In [ ]:
tokenizer("굳ㅋ", add_special_tokens=True, truncation=False)

{'input_ids': [1, 28705, 237, 184, 182, 230, 136, 142], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
training_args = script_args.training_args

In [ ]:
train_dataset = create_datasets(tokenizer, script_args)

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

100%|██████████| 400/400 [00:00<00:00, 3135.12it/s]

The character to token ratio of the dataset is: 0.93


In [ ]:
print(len(train_dataset))

2000


- SFTTrainer
  - 모델 학습 과정을 커스터마이즈하고, 특히 PEFT(Progressive Effort Fine-Tuning) 방식이나 LoRA(Low-Rank Adaptation) 같은 고급 미세조정 기법을 적용할 때 유용

In [ ]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    eval_dataset=None,
    peft_config=peft_config,
    packing=script_args.packing,
    max_seq_length=script_args.seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
base_model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=Fa

In [ ]:
print_trainable_parameters(base_model)

trainable params: 20971520 || all params: 3773042688 || trainable%: 0.5558251452256031


시퀀스 길이 512의 경우
- 14.4 G / 15.0 G 사용
- 메모리 오버플로우 발생시 512보다 줄일 것

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:456: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


Step,Training Loss
50,1.879800
100,1.009600
150,0.912000
200,0.933600
250,0.894000
300,0.911000
350,0.838000
400,0.882400
450,0.849400
500,0.832100


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:642: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7a85b06a54b0> was reported to be 2000 (when accessing len(dataloader)), but 2001 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:642: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7a85b06a54b0> was reported to be 2000 (when accessing len(dataloader)), but 2002 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:642: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7a85b06a54b0> was reported to be 2000 (when accessing len(dataloader)), but 2003 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:642: UserWarning: Le

TrainOutput(global_step=2000, training_loss=0.6460097141265869, metrics={'train_runtime': 5363.1352, 'train_samples_per_second': 0.746, 'train_steps_per_second': 0.373, 'total_flos': 4.3816744648704e+16, 'train_loss': 0.6460097141265869, 'epoch': 2.0})

In [ ]:
script_args.training_args.output_dir

'/gdrive/MyDrive/undergraduateReasearcher/mistral-instruct-v0.2-results-nsmc-consecutive'

In [ ]:
trainer.save_model(script_args.training_args.output_dir)

# 6. 추론 테스트

앞서 실행한 prepare_sample_text의 프롬프트와 동일화

In [ ]:
from transformers import pipeline, TextStreamer

수정 후

In [ ]:
# 최대한 원본을 벗어나지 않게끔 수정하는 방안
instruction_prompt_template = "[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: 굳ㅋ [/INST]\n반응: 긍정"

prompt_template = """[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "{User}" [/INST]\n반응: """

default_system_msg = (
    "너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다."
)


허깅페이스에 업로드 되어있는 nsmc 데이터셋에서 상위 11개 직접 입력
https://huggingface.co/datasets/nsmc

In [ ]:
# 수정 후
evaluation_queries = load_dataset('nsmc', split='test[:1000]')

In [ ]:
def wrapper_generate(model, input_prompt):
    data = tokenizer(input_prompt, return_tensors="pt")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    input_ids = data.input_ids[..., :-1]
    # print(input_prompt, input_ids)
    with torch.no_grad():
        pred = model.generate(
            input_ids=input_ids.cuda(),
            streamer=streamer,
            use_cache=True,
            max_new_tokens=float('inf'),
            temperature=0.5
        )
    decoded_text = tokenizer.batch_decode(pred, skip_special_tokens=True)
    print(decoded_text)
    return (decoded_text[0][len(input_prompt):])

In [ ]:
eval_dic = {
    i: (query, wrapper_generate(model=base_model, input_prompt=prompt_template.format(User=query['document'])))
    for i, query in enumerate(evaluation_queries.select(range(10)))
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"긍정" 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "굳 ㅋ" [/INST]\n반응: "긍정" ']
"긍정" 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

['[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "진정한 쓰레기" [/INST]\n반응: "부정" ']
"부정" 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다" [/INST]\n반응: "부정" ']
"부정" 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다" [/INST]\n반응: "부정" ']
"긍정" 
['[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.." [/INST]\n반응: "긍정" ']


In [ ]:
eval_dic

{0: ({'id': '6270596', 'document': '굳 ㅋ', 'label': 1}, '"긍정" '),
 1: ({'id': '9274899', 'document': 'GDNTOPCLASSINTHECLUB', 'label': 0},
  '"긍정" '),
 2: ({'id': '8544678',
   'document': '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아',
   'label': 0},
  '"부정" '),
 3: ({'id': '6825595',
   'document': '지루하지는 않은데 완전 막장임... 돈주고 보기에는....',
   'label': 0},
  '"부정" '),
 4: ({'id': '6723715',
   'document': '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??',
   'label': 0},
  '"부정" '),
 5: ({'id': '7898805', 'document': '음악이 주가 된, 최고의 음악영화', 'label': 1}, '"긍정" '),
 6: ({'id': '6315043', 'document': '진정한 쓰레기', 'label': 0}, '"부정" '),
 7: ({'id': '6097171',
   'document': '마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다',
   'label': 0},
  '"부정" '),
 8: ({'id': '8932678',
   'document': '갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다',
   'label': 0},
  '"부정" '),
 9: ({'id': '6242223',
   'document': '이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..',
   'label': 1},
  '"긍정" ')}

In [ ]:
eval_dic = {
    i: (query, wrapper_generate(model=base_model, input_prompt=prompt_template.format(User=query['document'])))
    for i, query in enumerate(evaluation_queries)
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"긍정" 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "굳 ㅋ" [/INST]\n반응: "긍정" ']
"긍정" 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

['[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "어색,유치. 소재는 좋았지만 결정적으로 재미가 없으니.." [/INST]\n반응: "부정" ']
"부정" 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "디젤, 개똥폼만잡고 디아블로란 존재도 멋지지도않고" [/INST]\n반응: "부정" ']
"부정" 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "밋밋하다고밖에 할말이 없다" [/INST]\n반응: "부정" ']
"부정" 
['[INST] 너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다. 리뷰: "에로용으로도 못 쓰는 영화" [/INST]\n반응: "부정" ']


In [ ]:
print(eval_dic[0])

({'id': '6270596', 'document': '굳 ㅋ', 'label': 1}, '"긍정" ')


In [ ]:
eval_dic

{0: ({'id': '6270596', 'document': '굳 ㅋ', 'label': 1}, '"긍정" '),
 1: ({'id': '9274899', 'document': 'GDNTOPCLASSINTHECLUB', 'label': 0},
  '"긍정" '),
 2: ({'id': '8544678',
   'document': '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아',
   'label': 0},
  '"부정" '),
 3: ({'id': '6825595',
   'document': '지루하지는 않은데 완전 막장임... 돈주고 보기에는....',
   'label': 0},
  '"부정" '),
 4: ({'id': '6723715',
   'document': '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??',
   'label': 0},
  '"부정" '),
 5: ({'id': '7898805', 'document': '음악이 주가 된, 최고의 음악영화', 'label': 1}, '"긍정" '),
 6: ({'id': '6315043', 'document': '진정한 쓰레기', 'label': 0}, '"부정" '),
 7: ({'id': '6097171',
   'document': '마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다',
   'label': 0},
  '"부정" '),
 8: ({'id': '8932678',
   'document': '갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다',
   'label': 0},
  '"부정" '),
 9: ({'id': '6242223',
   'document': '이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..',
   'label': 1},
  '"긍정" '),


In [ ]:
import pickle

# 특정 디렉토리에 eval_dic.pkl로 저장
save_path = '/gdrive/MyDrive/undergraduateReasearcher/mistral-7B-instruct-v0.2-nsmc-eval-dict/base-model-eval-dict-pickle-v0.1'  # 전체 경로 지정
with open(save_path, 'wb') as f:
    pickle.dump(eval_dic, f)

In [ ]:
# 파일에서 eval_dic 불러오기(확인용)
with open(save_path, 'rb') as f:
    loaded_eval_dic = pickle.load(f)

# 불러온 딕셔너리 사용
print(loaded_eval_dic)

{0: ({'id': '6270596', 'document': '굳 ㅋ', 'label': 1}, '"긍정" '), 1: ({'id': '9274899', 'document': 'GDNTOPCLASSINTHECLUB', 'label': 0}, '"긍정" '), 2: ({'id': '8544678', 'document': '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', 'label': 0}, '"부정" '), 3: ({'id': '6825595', 'document': '지루하지는 않은데 완전 막장임... 돈주고 보기에는....', 'label': 0}, '"부정" '), 4: ({'id': '6723715', 'document': '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??', 'label': 0}, '"부정" '), 5: ({'id': '7898805', 'document': '음악이 주가 된, 최고의 음악영화', 'label': 1}, '"긍정" '), 6: ({'id': '6315043', 'document': '진정한 쓰레기', 'label': 0}, '"부정" '), 7: ({'id': '6097171', 'document': '마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다', 'label': 0}, '"부정" '), 8: ({'id': '8932678', 'document': '갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다', 'label': 0}, '"부정" '), 9: ({'id': '6242223', 'document': '이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..', 'label': 1}, '"긍정" '), 10: ({'id': '7462111', 'document': '괜찮네요오랜만포켓몬스터잼밌어요', 'lab